# CNES com Notificacoes no SINAN

 

### Libs

In [1]:
%run ../../config/bootstrap.py

In [2]:
from pathlib import Path
import pandas as pd
import re
import numpy as np
from datetime import datetime 
from utils import get_project_root, save_parquet_in_chunks  
import basedosdados as bd

In [3]:
project_root = get_project_root() 
#billing_id = 'ufrgs-ppgc-dengue'

In [4]:
#%load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100
%sql bigquery://ufrgs-ppgc-dengue

Connecting to 'bigquery://ufrgs-ppgc-dengue'

# Dicionário

## ⌛ Staging -> 🥉 Bronze


In [5]:
folder = project_root / "data/1_staging/sinan"
list(folder.glob("*"))


[WindowsPath('C:/Users/silma/OneDrive/Área de Trabalho/Projetos/dengue/data/1_staging/sinan/cnes_w_notifications_part_1.parquet')]

In [6]:
path = project_root / "data/1_staging/sinan/cnes_w_notifications_part_1.parquet"
stg_df_filter = (
    pd.read_parquet(path, columns=["id_estabelecimento"])
      .rename(columns={"id_estabelecimento": "CO_CNES"})
      .drop_duplicates()
)

# Conferindo
stg_df_filter.info()
stg_df_filter.head(5).T


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77895 entries, 0 to 77894
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CO_CNES  77894 non-null  object
dtypes: object(1)
memory usage: 608.7+ KB


,0,1,2,3,4
CO_CNES,4013484,2589648,2052474,6270093,2589613


In [7]:
stg_df = pd.read_parquet(project_root / "data/1_staging/cnes/tbEstabelecimento", engine="pyarrow")
stg_df = stg_df.drop_duplicates()
stg_df.info()
stg_df.head(3).T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581577 entries, 0 to 581576
Data columns (total 56 columns):
 #   Column                                       Non-Null Count   Dtype 
---  ------                                       --------------   ----- 
 0   CO_UNIDADE                                   581577 non-null  object
 1   CO_CNES                                      581577 non-null  object
 2   NU_CNPJ_MANTENEDORA                          116895 non-null  object
 3   TP_PFPJ                                      581577 non-null  object
 4   NIVEL_DEP                                    581577 non-null  object
 5   NO_RAZAO_SOCIAL                              581576 non-null  object
 6   NO_FANTASIA                                  581509 non-null  object
 7   NO_LOGRADOURO                                581575 non-null  object
 8   NU_ENDERECO                                  581261 non-null  object
 9   NO_COMPLEMENTO                               283034 non-null  object
 

,0,1,2
CO_UNIDADE,2609602569302,2609602571943,2609602344637
CO_CNES,2569302,2571943,2344637
NU_CNPJ_MANTENEDORA,10404184000109,10404184000109,10404184000109
TP_PFPJ,3,3,3
NIVEL_DEP,3,3,3
NO_RAZAO_SOCIAL,PREFEITURA MUNICIPAL DE OLINDA,PREFEITURA MUNICIPAL DE OLINDA,PREFEITURA MUNICIPAL DE OLINDA
NO_FANTASIA,USF BULTRINS MONTE II,UNIDADE MOVEL,USF ALTO DA MINA
NO_LOGRADOURO,RUA PREFEITO MANOEL REGUEIRA,RUA DO SOL,RUA AVENCA
NU_ENDERECO,540,311,49
NO_COMPLEMENTO,None,None,None


### CNES com notificacoes

In [8]:
stg_df_filter["CO_CNES"]   = stg_df_filter["CO_CNES"].astype("string").str.replace(r"\D","", regex=True).str.zfill(7)
stg_df["CO_CNES"] = stg_df["CO_CNES"].astype("string").str.replace(r"\D","", regex=True).str.zfill(7)
bronze_df = stg_df_filter.merge(stg_df, on="CO_CNES", how="inner")
bronze_df = bronze_df.drop_duplicates()

In [9]:
bronze_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73477 entries, 0 to 73495
Data columns (total 56 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   CO_CNES                                      73477 non-null  string
 1   CO_UNIDADE                                   73477 non-null  object
 2   NU_CNPJ_MANTENEDORA                          56008 non-null  object
 3   TP_PFPJ                                      73477 non-null  object
 4   NIVEL_DEP                                    73477 non-null  object
 5   NO_RAZAO_SOCIAL                              73477 non-null  object
 6   NO_FANTASIA                                  73477 non-null  object
 7   NO_LOGRADOURO                                73477 non-null  object
 8   NU_ENDERECO                                  73464 non-null  object
 9   NO_COMPLEMENTO                               18277 non-null  object
 10  NO_BAIRRO      

In [10]:
output = project_root / "data/2_bronze/cnes/tbEstabelecimento_w_Notifications/tbEstabelecimento_w_Notifications" 
save_parquet_in_chunks(bronze_df,output,50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/73477 [00:00<?, ?linhas/s]

✔️ tbEstabelecimento_w_Notifications_part_1.parquet salvo com 7.35 MB (73477 linhas)
✅ Todos os arquivos salvos com sucesso.


In [11]:
del(stg_df)
del(stg_df_filter)
del(bronze_df)

## 🥉 Bronze -> 🥈 Silver

In [41]:
bronze_df = pd.read_parquet(project_root / "data/2_bronze/cnes/tbEstabelecimento_w_Notifications", engine="pyarrow")
bronze_df = bronze_df.drop_duplicates()
bronze_df.info()
bronze_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73477 entries, 0 to 73476
Data columns (total 56 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   CO_CNES                                      73477 non-null  string
 1   CO_UNIDADE                                   73477 non-null  object
 2   NU_CNPJ_MANTENEDORA                          56008 non-null  object
 3   TP_PFPJ                                      73477 non-null  object
 4   NIVEL_DEP                                    73477 non-null  object
 5   NO_RAZAO_SOCIAL                              73477 non-null  object
 6   NO_FANTASIA                                  73477 non-null  object
 7   NO_LOGRADOURO                                73477 non-null  object
 8   NU_ENDERECO                                  73464 non-null  object
 9   NO_COMPLEMENTO                               18277 non-null  object
 10  NO_BAIRRO 

,CO_CNES,CO_UNIDADE,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO,CO_TIPO_ABRANGENCIA,ST_COWORKING
0,4013484,2408104013484,08241754000145,3,3,RIO GRANDE DO NORTE SECRETARIA DA SAUDE PUBLICA,HOSPITAL GISELDA TRIGUEIRO,CONEGO MONTE,110,None,QUINTAS,59037170,7,None,04,None,32327915,None,None,None,08241754010965,03,03,None,None,None,None,None,05,06,24,240810,07/08/2025,SAUDE,68051085353,4363,None,None,None,-5.794974298299524,-35.209068059921265,06/12/2021,4013484,1023,S,None,S,None,None,D,30/10/2003,006,009,S,None,N
1,2589648,5201402589648,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APDA GO,UPA BRASICON,RUA DAS ACASIA,S/N,ESQ C AV CHILE,RESIDENCIAL BRASICON,74975390,001,None,None,None,3545 9953,None,None,None,None,04,03,None,None,None,None,None,73,06,52,520140,26/07/2024,YASMIN,03258865124,None,None,None,None,-16.82760,-49.24209,26/07/2024,YASMIN,1031,S,None,S,None,None,M,22/05/2002,008,007,None,None,None
2,2052474,3549802052474,None,3,1,UNIMED SJRPRETO COOPERATIVA DE TRABALHO MEDICO,COMPLEXO DE SAUDE UNIMED SAO JOSE DO RIO PRETO,AVENIDA BADY BASSITT,4870,None,JAD ALTO RIO PRETO,15025000,0215,None,None,None,17 3202-1110,None,unimed@unimedripreto.com.br,None,45100138000362,04,03,354980501-861-001673-1-2,13-set-2024 00:00:00,2,13-set-2025 00:00:00,1,73,06,35,354980,04/06/2025,FERNANDA,25123834826,None,None,None,None,-20.819074142767942,-49.397743074317205,01/04/2025,FERNANDA,2143,S,None,S,None,None,M,25/02/2003,008,007,N,None,None


### Cols renamed

In [59]:
dates = 'dt|data'
colunas_encontradas = bronze_df.columns[bronze_df.columns.str.contains(dates, case=False, regex=True)]
print(colunas_encontradas.tolist())


['dt_expedicao', 'dt_val_lic_sani', "to_char(dt_atualizacao,'dd/mm/yyyy')", "to_char(dt_atu_geo,'dd/mm/yyyy')", "to_char(dt_atualizacao_origem,'dd/mm/yyyy')"]


In [60]:
bronze_df.columns = bronze_df.columns.str.lower()

silver_df = bronze_df.rename(columns={
    "to_char(dt_atualizacao,'dd/mm/yyyy')": "data_atualizacao",
    "to_char(dt_atu_geo,'dd/mm/yyyy')": "data_atualizacao_geolocalizacao",
    "to_char(dt_atualizacao_origem,'dd/mm/yyyy')": "data_atualizacao_origem" ,
    "dt_expedicao": "data_expedicao",
    "dt_val_lic_sani": "data_val_lic_sani",
    "co_unidade": "id_unidade_cnes"
}
)

In [61]:
dates = 'dt|data'
colunas_encontradas = silver_df.columns[silver_df.columns.str.contains(dates, case=False, regex=True)]
print(colunas_encontradas.tolist())

['data_expedicao', 'data_val_lic_sani', 'data_atualizacao', 'data_atualizacao_geolocalizacao', 'data_atualizacao_origem']


### Dates

In [62]:
date_cols = ['data_expedicao', 'data_val_lic_sani', 'data_atualizacao', 'data_atualizacao_geolocalizacao', 'data_atualizacao_origem']
silver_df[colunas_encontradas].head()

,data_expedicao,data_val_lic_sani,data_atualizacao,data_atualizacao_geolocalizacao,data_atualizacao_origem
0,None,None,07/08/2025,06/12/2021,30/10/2003
1,None,None,26/07/2024,26/07/2024,22/05/2002
2,13-set-2024 00:00:00,13-set-2025 00:00:00,04/06/2025,01/04/2025,25/02/2003
3,23-fev-2018 00:00:00,23-fev-2019 00:00:00,08/08/2025,01/10/2019,23/06/2009
4,31-jan-2024 00:00:00,31-dez-2024 00:00:00,28/01/2025,01/02/2024,22/05/2002


In [63]:
meses_pt = {
    'jan': '01',
    'fev': '02',
    'mar': '03',
    'abr': '04',
    'mai': '05',
    'jun': '06',
    'jul': '07',
    'ago': '08',
    'set': '09',
    'out': '10',
    'nov': '11',
    'dez': '12'
}

def meses_para_numeros(texto):
    if pd.isna(texto):
        return texto
    texto = texto.lower()
    # Substitui abreviação do mês por número
    for mes, num in meses_pt.items():
        # corresponde apenas ao mês isolado (ex: 13-set-2024)
        texto = re.sub(rf'\b{mes}\b', num, texto)
    return texto
 
for col in date_cols:
    # Substitui meses
    silver_df[col] = silver_df[col].apply(meses_para_numeros)
    # Converte para datetime
    silver_df[col] = pd.to_datetime(silver_df[col], dayfirst=True, errors='coerce')
    # Formata para yyyy-mm-dd HH:00:00
    silver_df[col] = silver_df[col].dt.strftime('%Y-%m-%d %H:00:00')

silver_df[date_cols].head()


,data_expedicao,data_val_lic_sani,data_atualizacao,data_atualizacao_geolocalizacao,data_atualizacao_origem
0,NaN,NaN,2025-08-07 00:00:00,2021-12-06 00:00:00,2003-10-30 00:00:00
1,NaN,NaN,2024-07-26 00:00:00,2024-07-26 00:00:00,2002-05-22 00:00:00
2,2024-09-13 00:00:00,2025-09-13 00:00:00,2025-06-04 00:00:00,2025-04-01 00:00:00,2003-02-25 00:00:00
3,2018-02-23 00:00:00,2019-02-23 00:00:00,2025-08-08 00:00:00,2019-10-01 00:00:00,2009-06-23 00:00:00
4,2024-01-31 00:00:00,2024-12-31 00:00:00,2025-01-28 00:00:00,2024-02-01 00:00:00,2002-05-22 00:00:00


### Geolocation

In [64]:
silver_df[['nu_latitude','nu_longitude']].head()

,nu_latitude,nu_longitude
0,-5.794974298299524,-35.209068059921265
1,-16.82760,-49.24209
2,-20.819074142767942,-49.397743074317205
3,-20.777456,-49.403220
4,-16.7474832,-49.2812776


In [65]:
print(silver_df[['nu_latitude','nu_longitude']].describe())

       nu_latitude nu_longitude
count        69561        69563
unique       65537        65559
top        -23.548      -46.636
freq           115          115


In [66]:
geolocation = ['nu_latitude','nu_longitude']

for col in geolocation:
    # Se as colunas forem numéricas armazenadas como string ou objetos
    silver_df[col] = silver_df[col].astype(str).str.replace('.', ',', regex=False)

# Conferindo
print(silver_df[geolocation].head())


           nu_latitude         nu_longitude
0   -5,794974298299524  -35,209068059921265
1            -16,82760            -49,24209
2  -20,819074142767942  -49,397743074317205
3           -20,777456           -49,403220
4          -16,7474832          -49,2812776


### tp = type cols

In [67]:
tipo_pattern = r'tp|tipo|co_'
type_cols = silver_df.columns[silver_df.columns.str.contains(tipo_pattern, case=False, regex=True)]
print(type_cols.tolist())

['co_cnes', 'tp_pfpj', 'co_cep', 'co_regiao_saude', 'co_micro_regiao', 'co_distrito_sanitario', 'co_distrito_administrativo', 'co_atividade', 'co_clientela', 'tp_orgao_expedidor', 'tp_lic_sani', 'tp_unidade', 'co_turno_atendimento', 'co_estado_gestor', 'co_municipio_gestor', 'co_usuario', 'co_cpfdiretorcln', 'co_motivo_desab', 'co_natureza_jur', 'tp_estab_sempre_aberto', 'co_tipo_unidade', 'tp_gestao', 'co_tipo_estabelecimento', 'co_atividade_principal', 'co_tipo_abrangencia']


In [68]:
# Lista de colunas que você quer renomear
cols_to_rename = [
    'co_cnes', 'co_unidade', 'tp_pfpj', 'co_cep', 'co_regiao_saude', 
    'co_micro_regiao', 'co_distrito_sanitario', 'co_distrito_administrativo', 
    'co_atividade', 'co_clientela', 'tp_orgao_expedidor', 'tp_lic_sani', 
    'tp_unidade', 'co_turno_atendimento', 'co_estado_gestor', 'co_municipio_gestor', 
    'co_usuario', 'co_cpfdiretorcln', 'co_motivo_desab', 'co_natureza_jur', 
    'tp_estab_sempre_aberto', 'co_tipo_unidade', 'tp_gestao', 'co_tipo_estabelecimento', 
    'co_atividade_principal', 'co_tipo_abrangencia'
]

# Cria o dicionário de mapeamento substituindo 'co_' ou 'tp_' por 'id_'
rename_dict = {col: col.replace('co_', 'id_').replace('tp_', 'id_') for col in cols_to_rename}

# Aplica o rename no DataFrame
silver_df = silver_df.rename(columns=rename_dict)

# Confere o resultado
silver_df[rename_dict.values()].head()



,id_cnes,id_unidade,id_pfpj,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,id_atividade,id_clientela,id_orgao_expedidor,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,id_usuario,id_cpfdiretorcln,id_motivo_desab,id_natureza_jur,id_estab_sempre_aberto,id_tipo_unidade,id_gestao,id_tipo_estabelecimento,id_atividade_principal,id_tipo_abrangencia
0,4013484,05,3,59037170,7,None,04,None,03,03,None,None,05,06,24,240810,SAUDE,68051085353,None,1023,S,None,D,006,009,None
1,2589648,73,3,74975390,001,None,None,None,04,03,None,None,73,06,52,520140,YASMIN,03258865124,None,1031,S,None,M,008,007,None
2,2052474,73,3,15025000,0215,None,None,None,04,03,2,1,73,06,35,354980,FERNANDA,25123834826,None,2143,S,None,M,008,007,None
3,6270093,73,3,15047050,0215,None,None,None,03,03,2,1,73,06,35,354980,FERNANDA,21728064805,None,1244,S,None,M,008,007,None
4,2589613,20,3,74916970,001,None,None,None,04,03,2,1,20,06,52,520140,YASMIN,01162515147,None,1031,S,None,M,008,007,None


In [69]:
tipo_pattern = r'id_'
type_cols = silver_df.columns[silver_df.columns.str.contains(tipo_pattern, case=False, regex=True)]
print(type_cols.tolist())

['id_cnes', 'id_unidade_cnes', 'id_pfpj', 'id_cep', 'id_regiao_saude', 'id_micro_regiao', 'id_distrito_sanitario', 'id_distrito_administrativo', 'id_atividade', 'id_clientela', 'id_orgao_expedidor', 'id_lic_sani', 'id_unidade', 'id_turno_atendimento', 'id_estado_gestor', 'id_municipio_gestor', 'id_usuario', 'id_cpfdiretorcln', 'id_motivo_desab', 'id_natureza_jur', 'id_estab_sempre_aberto', 'id_tipo_unidade', 'id_gestao', 'id_tipo_estabelecimento', 'id_atividade_principal', 'id_tipo_abrangencia']


In [70]:
type_cols = ['id_pfpj', 'id_regiao_saude', 'id_micro_regiao', 'id_distrito_sanitario', 'id_distrito_administrativo', 'id_atividade', 'id_clientela', 'id_orgao_expedidor', 'id_lic_sani', 'id_turno_atendimento', 'id_estado_gestor', 'id_municipio_gestor', 'id_usuario', 'id_cpfdiretorcln', 'id_motivo_desab', 'id_natureza_jur', 'id_estab_sempre_aberto', 'id_tipo_unidade', 'id_gestao', 'id_tipo_estabelecimento', 'id_atividade_principal', 'id_tipo_abrangencia']
dfs = []

for col in type_cols:
    vc = silver_df[col].value_counts(dropna=False).reset_index()
    vc.columns = ['valor', 'qt']
    vc['coluna'] = col
    dfs.append(vc)  # apenas adiciona na lista, não concatena ainda

# Agora concatena todos em um único DataFrame
df_tipos = pd.concat(dfs, ignore_index=True)[['coluna', 'valor', 'qt']]

df_tipos.head()


,coluna,valor,qt
0,id_pfpj,3,71398
1,id_pfpj,1,2079
2,id_regiao_saude,None,22886
3,id_regiao_saude,001,2256
4,id_regiao_saude,0001,1394


"id" 

### hash

In [71]:
scd2_cols = silver_df.columns.tolist()

import hashlib

def row_hash_sha256(row):
    # Concatena todas as colunas como string separadas por '|', para garantir consistência
    row_str = '|'.join(row.astype(str).fillna(''))
    # Retorna o hash SHA-256
    return hashlib.sha256(row_str.encode('utf-8')).hexdigest()

# Cria a coluna 'hash'
silver_df['hash'] = silver_df.apply(row_hash_sha256, axis=1)


In [ ]:
se df_dia_n.hash<>df_bronze.hash
entao append df_dia_n.hash n df_bronze

In [72]:
from datetime import datetime

now = pd.Timestamp(datetime.now().date())  # só a data

silver_df['data_inicio_vigencia_silver'] = now
silver_df['data_fim_vigencia_silver'] = pd.Timestamp('9999-12-31')
silver_df['data_atualizacao_silver'] = now
silver_df['vigencia_silver'] = 1
silver_df.head()

,id_cnes,id_unidade_cnes,nu_cnpj_mantenedora,id_pfpj,nivel_dep,no_razao_social,no_fantasia,no_logradouro,nu_endereco,no_complemento,no_bairro,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,nu_telefone,nu_fax,no_email,nu_cpf,nu_cnpj,id_atividade,id_clientela,nu_alvara,data_expedicao,id_orgao_expedidor,data_val_lic_sani,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,data_atualizacao,id_usuario,id_cpfdiretorcln,reg_diretorcln,st_adesao_filantrop,id_motivo_desab,no_url,nu_latitude,nu_longitude,data_atualizacao_geolocalizacao,no_usuario_geo,id_natureza_jur,id_estab_sempre_aberto,st_geracredito_gerente_sgif,st_conexao_internet,id_tipo_unidade,no_fantasia_abrev,id_gestao,data_atualizacao_origem,id_tipo_estabelecimento,id_atividade_principal,st_contrato_formalizado,id_tipo_abrangencia,st_coworking,hash,data_inicio_vigencia_silver,data_fim_vigencia_silver,data_atualizacao_silver,vigencia_silver
0,4013484,2408104013484,08241754000145,3,3,RIO GRANDE DO NORTE SECRETARIA DA SAUDE PUBLICA,HOSPITAL GISELDA TRIGUEIRO,CONEGO MONTE,110,None,QUINTAS,59037170,7,None,04,None,32327915,None,None,None,08241754010965,03,03,None,NaN,None,NaN,None,05,06,24,240810,2025-08-07 00:00:00,SAUDE,68051085353,4363,None,None,None,"-5,794974298299524","-35,209068059921265",2021-12-06 00:00:00,4013484,1023,S,None,S,None,None,D,2003-10-30 00:00:00,006,009,S,None,N,4c98b01f2fd50a2da5a7fcb49580d51674ee156797054d93dc8b3b301afade8b,2025-08-17,9999-12-31,2025-08-17,1
1,2589648,5201402589648,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APDA GO,UPA BRASICON,RUA DAS ACASIA,S/N,ESQ C AV CHILE,RESIDENCIAL BRASICON,74975390,001,None,None,None,3545 9953,None,None,None,None,04,03,None,NaN,None,NaN,None,73,06,52,520140,2024-07-26 00:00:00,YASMIN,03258865124,None,None,None,None,"-16,82760","-49,24209",2024-07-26 00:00:00,YASMIN,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,None,None,None,f70e972e8d235757c25415e536078cb9b4e8e1b93b4d07280c2e961b8154df77,2025-08-17,9999-12-31,2025-08-17,1
2,2052474,3549802052474,None,3,1,UNIMED SJRPRETO COOPERATIVA DE TRABALHO MEDICO,COMPLEXO DE SAUDE UNIMED SAO JOSE DO RIO PRETO,AVENIDA BADY BASSITT,4870,None,JAD ALTO RIO PRETO,15025000,0215,None,None,None,17 3202-1110,None,unimed@unimedripreto.com.br,None,45100138000362,04,03,354980501-861-001673-1-2,2024-09-13 00:00:00,2,2025-09-13 00:00:00,1,73,06,35,354980,2025-06-04 00:00:00,FERNANDA,25123834826,None,None,None,None,"-20,819074142767942","-49,397743074317205",2025-04-01 00:00:00,FERNANDA,2143,S,None,S,None,None,M,2003-02-25 00:00:00,008,007,N,None,None,66339a46703b49c1bab3c25113d169321010c222f882e1944b11b9d5363cae9d,2025-08-17,9999-12-31,2025-08-17,1
3,6270093,3549806270093,46588950000180,3,3,PREFEITURA MUNICIPAL DE SAO JOSE DO RIO PRETO,UPA REGIAO NORTE,R MANOEL MORENO ESQ R JOSEPHA VOLTARELLI SANFELICE,130,None,JARDIM ANTUNES,15047050,0215,None,None,None,17-32375315,None,None,None,None,03,03,354980501-861-002357-1-7,2018-02-23 00:00:00,2,2019-02-23 00:00:00,1,73,06,35,354980,2025-08-08 00:00:00,FERNANDA,21728064805,473613,None,None,None,"-20,777456","-49,403220",2019-10-01 00:00:00,FERNANDA,1244,S,None,S,None,None,M,2009-06-23 00:00:00,008,007,None,None,None,245706442b1794984d4002593ef3ea47d4e082ca8ca5a07ffbe07421cdc06207,2025-08-17,9999-12-31,2025-08-17,1
4,2589613,5201402589613,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APARECIDA DE GOIANIA,CAIS NOVA ERA,AVENIDA SAO JOAO,S/N,None,JARDIM NOVA ERA,74916970,001,None,None,None,(62)3545 5925,None,None,None,None,04,03,None,2024-01-31 00:00:00,2,2024-12-31 00:00:00,1,20,06,52,520140,2025-01-28 00:00:00,YASMIN,01162515147,None,None,None,None,"-16,7474832","-49,2812776",2024-02-01 00:00:00,MARCIA,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,S,None,None,603e683c06ba7c1292ed6742e66b4bbc3e50da43cafb25e931b6f944e1324839,2025-08-17,9999-12-31,2025-08-17,1


In [73]:
silver_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73477 entries, 0 to 73476
Data columns (total 61 columns):
 #   Column                           Non-Null Count  Dtype        
---  ------                           --------------  -----        
 0   id_cnes                          73477 non-null  string       
 1   id_unidade_cnes                  73477 non-null  object       
 2   nu_cnpj_mantenedora              56008 non-null  object       
 3   id_pfpj                          73477 non-null  object       
 4   nivel_dep                        73477 non-null  object       
 5   no_razao_social                  73477 non-null  object       
 6   no_fantasia                      73477 non-null  object       
 7   no_logradouro                    73477 non-null  object       
 8   nu_endereco                      73464 non-null  object       
 9   no_complemento                   18277 non-null  object       
 10  no_bairro                        73477 non-null  object       
 11  id

## save

In [74]:
# Colunas duplicadas
dupes = silver_df.columns[silver_df.columns.duplicated()].tolist()
print(dupes)


[]


In [75]:
output = project_root / "data/3_silver/cnes/hst_dim_cnes/hst_dim_cnes" 
save_parquet_in_chunks(silver_df,output,50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/73477 [00:00<?, ?linhas/s]

# 🥇 Gold

In [83]:
path = project_root / "data/3_silver/cnes/hst_dim_cnes/hst_dim_cnes_part_1.parquet"
gold_df = pd.read_parquet(path)
gold_df = gold_df[gold_df['vigencia_silver'] == 1]
gold_df.head()

,id_cnes,id_unidade_cnes,nu_cnpj_mantenedora,id_pfpj,nivel_dep,no_razao_social,no_fantasia,no_logradouro,nu_endereco,no_complemento,no_bairro,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,nu_telefone,nu_fax,no_email,nu_cpf,nu_cnpj,id_atividade,id_clientela,nu_alvara,data_expedicao,id_orgao_expedidor,data_val_lic_sani,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,data_atualizacao,id_usuario,id_cpfdiretorcln,reg_diretorcln,st_adesao_filantrop,id_motivo_desab,no_url,nu_latitude,nu_longitude,data_atualizacao_geolocalizacao,no_usuario_geo,id_natureza_jur,id_estab_sempre_aberto,st_geracredito_gerente_sgif,st_conexao_internet,id_tipo_unidade,no_fantasia_abrev,id_gestao,data_atualizacao_origem,id_tipo_estabelecimento,id_atividade_principal,st_contrato_formalizado,id_tipo_abrangencia,st_coworking,hash,data_inicio_vigencia_silver,data_fim_vigencia_silver,data_atualizacao_silver,vigencia_silver
0,4013484,2408104013484,08241754000145,3,3,RIO GRANDE DO NORTE SECRETARIA DA SAUDE PUBLICA,HOSPITAL GISELDA TRIGUEIRO,CONEGO MONTE,110,None,QUINTAS,59037170,7,None,04,None,32327915,None,None,None,08241754010965,03,03,None,None,None,None,None,05,06,24,240810,2025-08-07 00:00:00,SAUDE,68051085353,4363,None,None,None,"-5,794974298299524","-35,209068059921265",2021-12-06 00:00:00,4013484,1023,S,None,S,None,None,D,2003-10-30 00:00:00,006,009,S,None,N,4c98b01f2fd50a2da5a7fcb49580d51674ee156797054d93dc8b3b301afade8b,2025-08-17,9999-12-31,2025-08-17,1
1,2589648,5201402589648,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APDA GO,UPA BRASICON,RUA DAS ACASIA,S/N,ESQ C AV CHILE,RESIDENCIAL BRASICON,74975390,001,None,None,None,3545 9953,None,None,None,None,04,03,None,None,None,None,None,73,06,52,520140,2024-07-26 00:00:00,YASMIN,03258865124,None,None,None,None,"-16,82760","-49,24209",2024-07-26 00:00:00,YASMIN,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,None,None,None,f70e972e8d235757c25415e536078cb9b4e8e1b93b4d07280c2e961b8154df77,2025-08-17,9999-12-31,2025-08-17,1
2,2052474,3549802052474,None,3,1,UNIMED SJRPRETO COOPERATIVA DE TRABALHO MEDICO,COMPLEXO DE SAUDE UNIMED SAO JOSE DO RIO PRETO,AVENIDA BADY BASSITT,4870,None,JAD ALTO RIO PRETO,15025000,0215,None,None,None,17 3202-1110,None,unimed@unimedripreto.com.br,None,45100138000362,04,03,354980501-861-001673-1-2,2024-09-13 00:00:00,2,2025-09-13 00:00:00,1,73,06,35,354980,2025-06-04 00:00:00,FERNANDA,25123834826,None,None,None,None,"-20,819074142767942","-49,397743074317205",2025-04-01 00:00:00,FERNANDA,2143,S,None,S,None,None,M,2003-02-25 00:00:00,008,007,N,None,None,66339a46703b49c1bab3c25113d169321010c222f882e1944b11b9d5363cae9d,2025-08-17,9999-12-31,2025-08-17,1
3,6270093,3549806270093,46588950000180,3,3,PREFEITURA MUNICIPAL DE SAO JOSE DO RIO PRETO,UPA REGIAO NORTE,R MANOEL MORENO ESQ R JOSEPHA VOLTARELLI SANFELICE,130,None,JARDIM ANTUNES,15047050,0215,None,None,None,17-32375315,None,None,None,None,03,03,354980501-861-002357-1-7,2018-02-23 00:00:00,2,2019-02-23 00:00:00,1,73,06,35,354980,2025-08-08 00:00:00,FERNANDA,21728064805,473613,None,None,None,"-20,777456","-49,403220",2019-10-01 00:00:00,FERNANDA,1244,S,None,S,None,None,M,2009-06-23 00:00:00,008,007,None,None,None,245706442b1794984d4002593ef3ea47d4e082ca8ca5a07ffbe07421cdc06207,2025-08-17,9999-12-31,2025-08-17,1
4,2589613,5201402589613,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APARECIDA DE GOIANIA,CAIS NOVA ERA,AVENIDA SAO JOAO,S/N,None,JARDIM NOVA ERA,74916970,001,None,None,None,(62)3545 5925,None,None,None,None,04,03,None,2024-01-31 00:00:00,2,2024-12-31 00:00:00,1,20,06,52,520140,2025-01-28 00:00:00,YASMIN,01162515147,None,None,None,None,"-16,7474832","-49,2812776",2024-02-01 00:00:00,MARCIA,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,S,None,None,603e683c06ba7c1292ed6742e66b4bbc3e50da43cafb25e931b6f944e1324839,2025-08-17,9999-12-31,2025-08-17,1


In [84]:
output = project_root / "data/4_gold/cnes/dim_cnes/dim_cnes" 
save_parquet_in_chunks(gold_df,output,50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/73477 [00:00<?, ?linhas/s]

✔️ dim_cnes_part_1.parquet salvo com 9.92 MB (73477 linhas)
✅ Todos os arquivos salvos com sucesso.
